In [1]:
from data_collection import CurrentSeasonalFPLData, HistoricFPLData
features = ['points_per_game','influence','expected_goal_involvements','total_points','creativity','bonus','ict_index',]
"""
Possible feature names:
['can_transact', 'can_select', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event',
'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
'removed', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points',
'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'region',
'team_join_date', 'birth_date', 'has_temporary_code', 'opta_code', 'minutes', 'goals_scored', 'assists', 'clean_sheets',
'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence',
'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded',
'mng_win', 'mng_draw', 'mng_loss', 'mng_underdog_win', 'mng_underdog_draw', 'mng_clean_sheets', 'mng_goals_scored', 'influence_rank',
'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'threat_rank', 'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type',
'corners_and_indirect_freekicks_order', 'corners_and_indirect_freekicks_text', 'direct_freekicks_order', 'direct_freekicks_text', 'penalties_order',
'penalties_text', 'expected_goals_per_90', 'saves_per_90', 'expected_assists_per_90', 'expected_goal_involvements_per_90',
'expected_goals_conceded_per_90', 'goals_conceded_per_90', 'now_cost_rank', 'now_cost_rank_type', 'form_rank', 'form_rank_type',
'points_per_game_rank', 'points_per_game_rank_type', 'selected_rank', 'selected_rank_type', 'starts_per_90', 'clean_sheets_per_90']
"""
data_getter = CurrentSeasonalFPLData(features=features)
test = HistoricFPLData(data_getter,lookback=3,future_points_weeks=3)

/home/zhillyz/fpl/data_collection.py:75: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[df.columns] = df[df.columns].apply(pandas.to_numeric,errors='ignore')
/home/zhillyz/fpl/data_collection.py:75: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[df.columns] = df[df.columns].apply(pandas.to_numeric,errors='ignore')
/home/zhillyz/fpl/data_collection.py:75: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[df.columns] = df[df.columns].apply(pandas.to_numeric,errors='ignore')
/home/zhillyz/fpl/data_collection.py:75: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and cat

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

X_train,Y_train = test.data
X_train, X_val, Y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=37)
model = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100))
model.fit(X_train, Y_train)

preds = model.predict(X_val)
print("MAE:", mean_absolute_error(y_val, preds))

MAE: 1.675446224256293


In [3]:
""" Run this for best Wildcard team."""

import numpy as np
import pandas as pd
from scipy.optimize import linprog

# Step 1: Predict Future Points for All Players
# Assuming 'model' is your trained model, and 'players_df' is a dataframe with player features
# Add predictions to your player dataframe
players_df = data_getter.processed_data.copy()
predict_data = players_df[data_getter.features]

# Ensure no NaN values in the features
predict_data = predict_data.fillna(0)

players_df['predicted_future_points'] = model.predict(predict_data)

# Step 2: Set Constraints

# Total budget (assuming the total budget is 100.0)
budget = 100  # Total budget in FPL units

# Player positions constraints (e.g., 2 goalkeepers, 5 defenders, 5 midfielders, 3 forwards)
position_constraints = {
    1:2,
    2:5,
    3:5,
    4:3,
}

# Assuming players_df has columns: 'id', 'position', 'cost', 'predicted_future_points'
# Step 3: Optimize the Team Selection

# Define cost and predicted points as arrays for optimization
costs = players_df['now_cost'].values / 10.0  # Convert cost from 0.1 million to actual cost (in millions)
future_points = players_df['predicted_future_points'].values
positions = players_df['element_type'].values

# Number of players to select (e.g., 15 players for an FPL team)
num_players = 15

# Define optimization coefficients (we want to maximize future points)
c = -future_points  # We negate because linprog minimizes by default

# Constraints:
# 1. Total cost must not exceed the budget
# 2. Positional constraints (e.g., 2 goalkeepers, 5 defenders, etc.)
A_eq = []
b_eq = []
for position, max_count in position_constraints.items():
    # Create a constraint matrix for each position
    position_indices = (positions == position)
    A_eq.append(position_indices.astype(int))  # Make sure the number of players for each position doesn't exceed max_count
    b_eq.append(max_count)

# 3. Select exactly 15 players
A_eq.append(np.ones(len(players_df)))  # Ensure we select exactly 15 players
b_eq.append(num_players)

# Step 4: Solve the optimization problem using linprog
result = linprog(c, A_eq=np.array(A_eq), b_eq=np.array(b_eq), bounds=[(0, 1)] * len(players_df), method='highs')

# Step 5: Check for errors and Get the selected players
if result.success:
    selected_player_indices = np.where(result.x >= 0.5)[0]  # Get the players with a value >= 0.5 (i.e., selected)

    # Step 6: Get the player IDs and their details
    selected_players = players_df.iloc[selected_player_indices]
    print(selected_players[['web_name', 'now_cost', 'element_type', 'predicted_future_points']])

    # Total cost of the selected players
    total_cost = selected_players['now_cost'].sum() / 10  # Convert cost from 0.1 million to actual cost
    total_points = selected_players['predicted_future_points'].sum()
    print(f"Total cost of selected players: {total_cost}")
    print(f"Total predicted points: {total_cost}")
else:
    print("Optimization failed: ", result.message)
    # Debugging: Check if total cost exceeds budget or if other issues exist
    print(f"Total cost of all players: {players_df['now_cost'].sum() / 10}")
    print(f"Number of players available: {len(players_df)}")
    print(f"Positional constraints: {position_constraints}")


        web_name now_cost element_type  predicted_future_points
74       O.Dango       48            3                     7.02
79        Senesi       46            2                     3.44
110        Wissa       65            4                     4.37
207       Mateta       78            4                     4.63
248         Leno       50            1                     6.03
117   Buonanotte       45            3                     5.59
326       Konaté       53            2                     7.24
625     Gündogan       63            3                     5.94
372     Garnacho       59            3                     4.74
389      Zirkzee       65            4                     4.01
392       Barnes       59            3                    10.85
409   Livramento       46            2                     3.61
443         Sels       51            1                     6.19
573   Milenković       51            2                     4.45
388  Wan-Bissaka       45            2  

/tmp/ipykernel_215853/3359898147.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predict_data = predict_data.fillna(0)


In [4]:
current_team_ids = [383,350,88,255,366,328,402,99,321,401,58,521,71,270,295]  # your 15-player team as player IDs
budget_left = 0.8        # remaining budget from original 1000 (or 100.0)

# Get your full player pool with predicted future points
all_players = players_df.copy()
all_players['predicted'] = model.predict(all_players[data_getter.features])

# Get your current team details
current_team = all_players[all_players['id'].isin(current_team_ids)]

best_transfer = None
best_gain = 0

# Loop through all current team players
for i, old_player in current_team.iterrows():
    old_pos = old_player['element_type']
    old_cost = old_player['now_cost']
    
    # Look at all possible replacements in same position
    candidates = all_players[
        (all_players['element_type'] == old_pos) &
        (~all_players['id'].isin(current_team_ids))  # not already owned
    ]
    
    for j, new_player in candidates.iterrows():
        cost_diff = new_player['now_cost'] - old_cost
        if cost_diff <= budget_left:
            # Create new team with the candidate transfer
            temp_team = current_team.copy()
            temp_team = temp_team[temp_team['id'] != old_player['id']]
            temp_team = pd.concat([temp_team, new_player.to_frame().T])
            
            team_points = temp_team['predicted'].sum()
            
            # Track the best one
            if team_points > best_gain:
                best_gain = team_points
                best_transfer = (old_player, new_player)
best_transfer

In [6]:
best_transfer

(id                                  71
 now_cost                            60
 element_type                         3
 web_name                      Kluivert
 points_per_game                    5.2
 influence                        680.4
 expected_goal_involvements       12.71
 total_points                       147
 creativity                       591.6
 bonus                               14
 ict_index                        196.6
 predicted_future_points           0.11
 predicted                         0.11
 Name: 71, dtype: object,
 id                               392
 now_cost                          59
 element_type                       3
 web_name                      Barnes
 points_per_game                  4.2
 influence                      464.2
 expected_goal_involvements      8.93
 total_points                     113
 creativity                     282.2
 bonus                             13
 ict_index                      128.0
 predicted_future_points        10.8